In [1]:
print('fhna')

fhna


In [1]:
import re
import json

def process_rule_part(rulepart, ftype, json_dta):
    """
    Process a single rule part from the GTM JSON data and return markdown text.
    
    Parameters:
        rulepart: A rule part, which can be a list (e.g. ["if", 0]) or even a scalar.
        ftype: The rule type identifier (e.g. "if", "unless", "add", "block")
        json_dta: The full JSON dictionary parsed from the GTM file.
    """
    output = ""
    # Lookup dictionaries (equivalent to PHP arrays)
    types_array = {
        "if": ["predicates", "(only if PREDICATE is TRUE)", ""],
        "unless": ["predicates", "(only if PREDICATE is FALSE)", "DOES NOT"],
        "add": ["tags", "FIRING"],
        "block": ["tags", "BLOCKING"],
    }
    
    tags = {
        "__googtag": "GA4 Google Tag",
        "__gaawe": "GA4 Event Tag (GA App and WEb)",
        "__img": "Image Tag Event",
        "__html": "HTML Tag",
        "__cl": "Element Click Event",
        "__lcl": "Link Click Event",
        "__sdl": "Scroll Event",
        "__evl": "Element Visibility Event",
        "__fsl": "Form Submit Event",
        "__ytl": "Youtube Event",
        "__tl": "Timer Event",
        "__tg": "Trigger Group",
        "__jel": "Javascript Error Event",
        "__hl": "History Event",
        "__cvt": "Custom Template Event",
        "__asprv": "Unknown? __asprv",
    }
    
    macro_functions = {
        "__aev": "auto event variable",
        "__awec": "User-Provided Data",
        "__c": "constant",
        "__cid": "Container ID",
        "__ctv": "Container Version Number",
        "__d": "dom element",
        "__dbg": "Debug",
        "__e": "Event",
        "__f": "HTTP Referrer",
        "__gas": "Google Analytics Settings",
        "__gtcs": "Google Tag Config Settings",
        "__gtes": "Google Tag Event Settings",
        "__j": "javascript variable",
        "__jsm": "custom javascript variable",
        "__k": "1st party cookie",
        "__r": "Random Number",
        "__remm": "regex lookup table",
        "__smm": "lookup table",
        "__u": "URL",
        "__uv": "Undefined Value",
        "__v": "data layer variable",
        "__vis": "Visibility",
    }
    
    predicates = {
        "_eq": "equals",
        "_re": "regex match",
        "_ew": "ends with",
        "_sw": "starts with",
        "_cn": "contains",
        "_lt": "less than",
        "_ge": "greater than or equal to",
        "_css": "matches CSS selector",
    }
    
    # ----- 1. Check if rulepart itself is a rule-type indicator  -----
    if isinstance(rulepart, list) and len(rulepart) > 0:
        # If the first element is a string with letters, assume it's the rule type.
        if isinstance(rulepart[0], str) and re.search(r"[a-z]+", rulepart[0]):
            rule_type = rulepart[0]
            # Color hint (informational only; markdown doesn't support colors)
            rule_color = "red" if rule_type == "if" else "blue"
            
            output += f"**rule type:** _[{rulepart}]_ (color hint: {rule_color})\n"
            return output

    # ----- 2. Process the rule further based on its type (predicates or tags) -----
    try:
        key_type = types_array.get(ftype, [None])[0]
        # Use rulepart as a key/index; note: rulepart might be an int.
        json_predicates = json_dta[key_type][rulepart]
    except Exception as e:
        output += f"Error retrieving json_predicates using key '{rulepart}': {e}\n"
        return output

    # ----- 3. Branch based on the ftype: tags vs. predicates -----
    if ftype in ["add", "block"]:
        # ---- TAGS branch ----
        try:
            whole_tag = json_dta["tags"][rulepart]
        except Exception as e:
            output += f"Error retrieving tag for key '{rulepart}': {e}\n"
            return output

        tag_function = whole_tag.get("function", "")
        tag_function_human = tags.get(tag_function, tag_function)
        
        event_name = f"**event_name:** {whole_tag['vtp_eventName']}\n" if "vtp_eventName" in whole_tag else ""
        tag_params_table = ""
        tag_macro_vals = ""
        vtp_table = whole_tag.get("vtp_eventSettingsTable")
        
        if vtp_table and isinstance(vtp_table, list) and len(vtp_table) > 0 and vtp_table[0] == "list":
            # Build a markdown table header using values from vtp_table
            try:
                header = f"| {vtp_table[1][1]} | {vtp_table[1][3]} |\n| --- | --- |\n"
            except Exception:
                header = "| Header error | Header error |\n| --- | --- |\n"
            rows = ""
            for tag_row in vtp_table:
                if isinstance(tag_row, list) and tag_row != "list" and len(tag_row) > 0 and tag_row[0] == "map":
                    # Check if the parameter value is defined via a macro
                    if isinstance(tag_row[4], list) and len(tag_row[4]) > 0 and tag_row[4][0] == "macro":
                        tag_macro_index = tag_row[4][1]
                        try:
                            tag_macro = json_dta["macros"][tag_macro_index]
                        except Exception as e:
                            tag_macro = {}
                        tag_macro_function = tag_macro.get("function", "")
                        tag_macro_vtp_name = f"\nname: {tag_macro['vtp_name']}" if "vtp_name" in tag_macro else ""
                        tag_macro_function_human = f"**{macro_functions.get(tag_macro_function, tag_macro_function)}**{tag_macro_vtp_name}\n"
                        tag_macro_function_human += f"\n{repr(tag_macro)}\n"
                        tag_macro_vals = ""
                        for k, v in tag_macro.items():
                            tag_macro_vals += f"{k} => {v}\n"
                    else:
                        tag_macro = tag_row[4]
                        tag_macro_function_human = f'static:"{tag_macro}"'
                        tag_macro_vals = ""
                    try:
                        param_key = tag_row[2]
                    except Exception:
                        param_key = "unknown"
                    rows += f"| {param_key} | {tag_macro_function_human} |\n"
            tag_params_table = header + rows
        else:
            tag_params_table = f"\n{repr(whole_tag)}\n"
            tag_macro_vals = "no macro vals"
        # Build an overview table of all tag details
        tag_rows_table = "| key | value |\n| --- | --- |\n"
        for key_item, value_item in whole_tag.items():
            if key_item == "vtp_eventSettingsTable":
                tag_rows_table += f"| Event Parameters ({key_item}) | {tag_params_table} |\n"
            else:
                tag_rows_table += f"| {key_item} | {value_item if isinstance(value_item, str) else repr(value_item)} |\n"

        output += f"### [TAG] type: {tag_function} - {tag_function_human}\n{event_name}\n"
        output += tag_rows_table + "\n"
        output += f"\n{tag_macro_vals}\n\n...________.....\n"
    else:
        # ---- PREDICATES branch (for ftype "if" or "unless") ----
        output += f"### ------ Begin {ftype} -------\n"
        output += f"{types_array[ftype][1]} #: [{rulepart}]\n"
        if isinstance(json_predicates.get("arg0"), list) and json_predicates["arg0"][0] == "macro":
            # If arg1 is also a macro, fetch its details
            if isinstance(json_predicates.get("arg1"), list) and json_predicates["arg1"][0] == "macro":
                submacros = json_dta["macros"][json_predicates["arg1"][1]]
                smacro_vals = ""
                for k, v in submacros.items():
                    smacro_vals += f"{k} => {v}\n"
                predicate_value = smacro_vals
            else:
                predicate_value = json_predicates.get("arg1", "")
            down_arrow = "↓"
            
            output += f"**[{ftype}]** function **{json_predicates.get('function', 'n/a')}**"
            output += f"(macro[{json_predicates['arg0'][1]}], \"{predicate_value}\")\n"

            
            output += down_arrow + "\n"
            macros = json_dta["macros"][json_predicates["arg0"][1]]
            macro_function = macros.get("function", "")
            macro_function_human = macro_functions.get(macro_function, macro_function)
            macro_index = json_predicates["arg0"][1]
            macro_vals = ""
            for k, v in macros.items():
                macro_vals += f"{k} => {v}"
            predicates_obj = repr(json_predicates)
            function_table = f"""
            
| 1 | 2 | 3 |
| ---- | -------------- | ---- |
| **type:** => | ... | {ftype} |
| **macro:** => | macro[{macro_index}] => {macro_vals} => | {macro_function_human} |
| **predicate:** => | {json_predicates.get('function','')} => | {predicates.get(json_predicates.get('function',''), json_predicates.get('function',''))} |
| **value:** => | ... | {predicate_value}|

aka:  {ftype} {macro_function_human} {predicates.get(json_predicates.get('function',''), json_predicates.get('function',''))} {predicate_value}

\n
{predicates_obj} \n\n

"""
            output += function_table + "\n"
        else:
            output += "\n--- NOT PREDICATES ---\n"
            try:
                output += str(json_predicates["arg0"][0]) + "\n"
            except Exception as e:
                output += "Error printing arg0: " + str(e) + "\n"
        output += f"\n### ------ End {ftype} -------\n"
    return output

def process_gtm_file(gtm_file_contents):
    """
    Extract the JSON object from the GTM script, iterate over its rules,
    and return all output as a markdown-formatted string.
    """
    output = ""
    match = re.search(r'"resource":\s*({.*?})\s*,\s*"runtime"', gtm_file_contents, re.DOTALL)
    if match:
        json_content = match.group(1)
        try:
            json_data = json.loads(json_content)
        except json.JSONDecodeError:
            output += "**Error:** Unable to parse JSON from GTM script.\n"
            return output
    else:
        output += "**Error:** Pattern not found in GTM file.\n"
        return output

    if "rules" in json_data:
        for rule in json_data["rules"]:
            for rpart in rule:
                for thisrp in rpart:
                    output += process_rule_part(thisrp, rpart[0], json_data)
    else:
        output += "**No rules found in the JSON data.**\n"
    return output

# Example usage:
with open("gtm.txt", "r", encoding="utf-8") as file:
    gtm_file_contents = file.read()

from IPython.display import display, Markdown
output = process_gtm_file(gtm_file_contents)
display(Markdown(output))


Error retrieving json_predicates using key 'if': list indices must be integers or slices, not str
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [0]
**[if]** function **_eq**(macro[0], "gtm.init")
↓

            
| 1 | 2 | 3 |
| ---- | -------------- | ---- |
| **type:** => | ... | if |
| **macro:** => | macro[0] => function => __e => | Event |
| **predicate:** => | _eq => | equals |
| **value:** => | ... | gtm.init|

aka:  if Event equals gtm.init



{'function': '_eq', 'arg0': ['macro', 0], 'arg1': 'gtm.init'} 





### ------ End if -------
Error retrieving json_predicates using key 'add': list indices must be integers or slices, not str
### [TAG] type: __googtag - GA4 Google Tag

| key | value |
| --- | --- |
| function | __googtag |
| metadata | ['map'] |
| once_per_event | True |
| vtp_tagId | G-V9YWX1TNDN |
| vtp_configSettingsTable | ['list', ['map', 'parameter', 'allow_ad_personalization_signals', 'parameterValue', '0']] |
| tag_id | 273 |


no macro vals

...________.....
Error retrieving json_predicates using key 'if': list indices must be integers or slices, not str
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [1]
**[if]** function **_css**(macro[1], "h1 a")
↓

            
| 1 | 2 | 3 |
| ---- | -------------- | ---- |
| **type:** => | ... | if |
| **macro:** => | macro[1] => function => __vvtp_name => gtm.elementvtp_dataLayerVersion => 1 => | data layer variable |
| **predicate:** => | _css => | matches CSS selector |
| **value:** => | ... | h1 a|

aka:  if data layer variable matches CSS selector h1 a



{'function': '_css', 'arg0': ['macro', 1], 'arg1': 'h1 a'} 





### ------ End if -------
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [2]
**[if]** function **_eq**(macro[0], "gtm.click")
↓

            
| 1 | 2 | 3 |
| ---- | -------------- | ---- |
| **type:** => | ... | if |
| **macro:** => | macro[0] => function => __e => | Event |
| **predicate:** => | _eq => | equals |
| **value:** => | ... | gtm.click|

aka:  if Event equals gtm.click



{'function': '_eq', 'arg0': ['macro', 0], 'arg1': 'gtm.click'} 





### ------ End if -------
Error retrieving json_predicates using key 'add': list indices must be integers or slices, not str
### [TAG] type: __gaawe - GA4 Event Tag (GA App and WEb)
**event_name:** header_click

| key | value |
| --- | --- |
| function | __gaawe |
| metadata | ['map'] |
| once_per_event | True |
| vtp_sendEcommerceData | False |
| vtp_enhancedUserId | False |
| vtp_eventName | header_click |
| vtp_measurementIdOverride | G-V9YWX1TNDN |
| vtp_enableUserProperties | True |
| vtp_enableMoreSettingsOption | True |
| vtp_enableEuid | True |
| vtp_migratedToV2 | True |
| vtp_demoV2 | False |
| tag_id | 275 |


no macro vals

...________.....
Error retrieving json_predicates using key 'if': list indices must be integers or slices, not str
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [3]
**[if]** function **_eq**(macro[0], "gtm.js")
↓

            
| 1 | 2 | 3 |
| ---- | -------------- | ---- |
| **type:** => | ... | if |
| **macro:** => | macro[0] => function => __e => | Event |
| **predicate:** => | _eq => | equals |
| **value:** => | ... | gtm.js|

aka:  if Event equals gtm.js



{'function': '_eq', 'arg0': ['macro', 0], 'arg1': 'gtm.js'} 





### ------ End if -------
Error retrieving json_predicates using key 'add': list indices must be integers or slices, not str
### [TAG] type: __cl - Element Click Event

| key | value |
| --- | --- |
| function | __cl |
| tag_id | 276 |


no macro vals

...________.....
### [TAG] type: __cl - Element Click Event

| key | value |
| --- | --- |
| function | __cl |
| tag_id | 277 |


no macro vals

...________.....
### [TAG] type: __cl - Element Click Event

| key | value |
| --- | --- |
| function | __cl |
| tag_id | 278 |


no macro vals

...________.....


In [21]:
with open("gtm.txt", "r", encoding="utf-8") as file:
    gtm_file_contents = file.read()


In [29]:
# Split the contents into individual lines
lines = gtm_file_contents.splitlines()

# Print the first 5 lines
for line in lines[:5]:
    print(line)



// Copyright 2012 Google Inc. All rights reserved.
 
(function(){



In [18]:
from IPython.display import display, Markdown
output = process_gtm_file(gtm_file_contents)
print(output)

Error retrieving json_predicates using key 'if': list indices must be integers or slices, not str
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [0]
**[if]** function **_eq**(macro[0], "gtm.init")
↓

            
**type:**    if

**macro:**   macro[0] =>  Event

**predicate:**    equals 

**value:** gtm.init




### ------ End if -------
Error retrieving json_predicates using key 'add': list indices must be integers or slices, not str
### [TAG] type: __googtag - GA4 Google Tag

| key | value |
| --- | --- |
| function | __googtag |
| metadata | ['map'] |
| once_per_event | True |
| vtp_tagId | G-V9YWX1TNDN |
| vtp_configSettingsTable | ['list', ['map', 'parameter', 'allow_ad_personalization_signals', 'parameterValue', '0']] |
| tag_id | 273 |


no macro vals

...________.....
Error retrieving json_predicates using key 'if': list indices must be integers or slices, not str
### ------ Begin if -------
(only if PREDICATE is TRUE) #: [1]
**[if]** function **_css**(macro[1], "h1